***Ratios***
- Standalone -> https://www.moneycontrol.com/financials/itc/ratiosVI/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/consolidated-ratiosVI/ITC#ITC

***Balance Sheet***
- Standalone -> https://www.moneycontrol.com/financials/itc/balance-sheetVI/ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC
- for next page https://www.moneycontrol.com/financials/itc/profit-lossVI/ITC/2#ITC

***Profit & Loss***- 
- Standalone -> https://www.moneycontrol.com/financials/itc/profit-lossVI/ITC
- Consolidated ->https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC#ITC

***Quarterly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/quarterly-results/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-quarterly-results/ITC#ITC

***Half Yearly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/half-yearly/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-half-yearly/ITC#ITC

***Nine Months Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/nine-months/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-nine-months/ITC#ITC

***Yearly Results***
- Standalone -> https://www.moneycontrol.com/financials/itc/results/yearly/ITC#ITC
- Consolidated -> https://www.moneycontrol.com/financials/itc/results/consolidated-yearly/ITC#ITC

***Cash Flow***
- Standalone - https://www.moneycontrol.com/financials/itc/cash-flowVI/ITC#ITC
- Consolidated - https://www.moneycontrol.com/financials/itc/consolidated-cash-flowVI/ITC#ITC

***Capital Structure***
- https://www.moneycontrol.com/financials/itc/capital-structure/ITC#ITC

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

***To download Financial ratios***

In [2]:
import pandas as pd
mc_fin_baseurl = 'https://www.moneycontrol.com/financials'

def processfindf(df):
    df.at[0,0] = 'Item'
    df = df.iloc[:, :-1]
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    df.set_index('Item', inplace=True)
    return df

# https://www.moneycontrol.com/financials/itc/consolidated-balance-sheetVI/ITC

# fintp -> 'ratios', 'balance-sheet', 'profit-loss', 'yearly', 'quarterly-results', 'half-yearly', 'nine-months', 'cash-flow'
def getfinancials(symbol, fintp='ratios'):
    symbol = symbol.upper()
    url = f'{mc_fin_baseurl}/{symbol}/consolidated-{fintp}VI/{symbol}'
    if fintp in ['yearly', 'quarterly-results', 'half-yearly', 'nine-months']:
        url = f'{mc_fin_baseurl}/{symbol}/results/consolidated-{fintp}/{symbol}'
    lst = [pd.read_html(f'{url}/{i}') for i in range(2, 6)]
    lst = [pd.read_html(url)] + lst
    dfs = [processfindf(i[0]) for i in lst if i]
    df = pd.concat(dfs, axis=1)
    df = df.dropna(how='all') 
    df.fillna(0, inplace=True)
    df = df[df.columns[::-1]]
    if fintp in ['balance-sheet', 'profit-loss']: df = df.iloc[1:]
    return df

In [52]:
%%time
symbol = 'HDF01'
ratios = getfinancials(symbol, 'ratios')
bs = getfinancials(symbol, 'balance-sheet')
pnl = getfinancials(symbol, 'profit-loss')
cashflow = getfinancials(symbol, 'cash-flow')
yearly = getfinancials(symbol, 'yearly')
half_yearly = getfinancials(symbol, 'half-yearly')
quarter = getfinancials(symbol, 'quarterly-results')
# nine_mnths = getfinancials(symbol, 'nine-months')

CPU times: user 10.7 s, sys: 310 ms, total: 11 s
Wall time: 1min 23s


In [53]:
company = 'HDFCBank'
excel_file = f'/Users/ajit/Google Drive/Fundemental Analysis/Financials_{company}.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    pnl.to_excel(writer, sheet_name='Income Statement')
    bs.to_excel(writer, sheet_name='Balance Sheet')
    cashflow.to_excel(writer, sheet_name='Cash Flow')
    ratios.to_excel(writer, sheet_name='Ratios')
    yearly.to_excel(writer, sheet_name='Yearly Results')
#     half_yearly.to_excel(writer, sheet_name='Half Yearly Results')
    quarter.to_excel(writer, sheet_name='Quarterly Results')
#     nine_mnths.to_excel(writer, sheet_name='9 Months Results')

In [45]:
import requests, json
response = requests.get('https://api.tickertape.in/stocks/financials/income/INFY/annual/normal?count=100')
data = response.json()['data']
df = pd.DataFrame(data)

In [46]:
cols = ['displayPeriod', 'incTrev', 'incPbt', 'incNinc', 'incEps', 'incDps', 'incPyr']
df = df[cols]
df.columns = ['period', 'revenue', 'pbt', 'income', 'eps', 'dps', 'payout_ratio']

In [48]:
df

,period,revenue,pbt,income,eps,dps,payout_ratio
0,FY 2012,35638,11699,8332,18.227345,5.875022,0.322319
1,FY 2013,42717,12799,9429,20.627104,5.250039,0.254521
2,FY 2014,52797,14728,10656,23.311250,7.875069,0.337823
3,FY 2015,56749,17284,12372,27.065100,14.875000,0.549601
4,FY 2016,65561,18740,13489,29.508608,12.125121,0.410901
5,FY 2017,71534,19951,14353,31.398391,12.875000,0.410053
6,FY 2018,73833,20270,16029,35.947784,21.750000,0.605044
7,FY 2019,85557,21041,15404,35.482545,21.500000,0.605932
8,FY 2020,93594,22007,16594,38.695501,17.500000,0.452249
